# 🔬 LMFDB Conductor Selectivity Test

**Real L-function zeros from LMFDB API**

Testing: Do GIFT conductors show better [5, 8, 13, 27] recurrence structure?

---

## GIFT vs Non-GIFT Conductors

- **GIFT**: {7, 8, 11, 13, 14, 21, 27, 77, 99}
- **Non-GIFT**: {6, 9, 10, 15, 16, 17, 19, 23, 25}

In [ ]:
# Install dependencies
!pip install requests numpy scipy matplotlib --quiet
print("✓ Dependencies installed")

In [ ]:
import requests
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import json
from typing import List, Dict, Tuple, Optional
import time

print("Libraries loaded")

## 1. LMFDB API Functions

In [ ]:
LMFDB_BASE = "https://www.lmfdb.org/api"

def get_dirichlet_characters(modulus: int) -> List[Dict]:
    """
    Get Dirichlet characters for a given modulus from LMFDB.
    """
    url = f"{LMFDB_BASE}/char_dir_orbits/?modulus={modulus}&_format=json"
    try:
        response = requests.get(url, timeout=30)
        if response.status_code == 200:
            data = response.json()
            return data.get('data', [])
    except Exception as e:
        print(f"Error fetching characters for modulus {modulus}: {e}")
    return []


def get_lfunc_zeros_by_label(label: str, max_zeros: int = 100) -> List[float]:
    """
    Get zeros of an L-function by its LMFDB label.
    
    For Dirichlet L-functions, labels look like: 'character/dirichlet/13/2'
    """
    # Try the L-function zeros endpoint
    url = f"{LMFDB_BASE}/lfunc_zeros/?origin={label}&_format=json"
    try:
        response = requests.get(url, timeout=30)
        if response.status_code == 200:
            data = response.json()
            zeros_data = data.get('data', [])
            if zeros_data:
                # Extract imaginary parts
                zeros = []
                for item in zeros_data[:max_zeros]:
                    if 'zeros' in item:
                        zeros.extend(item['zeros'])
                    elif 'zero' in item:
                        zeros.append(item['zero'])
                return sorted(zeros)[:max_zeros]
    except Exception as e:
        print(f"  Error fetching zeros for {label}: {e}")
    return []


def get_dirichlet_lfunc_zeros(modulus: int, char_index: int = 1, max_zeros: int = 100) -> List[float]:
    """
    Get zeros of Dirichlet L-function L(s, χ) where χ is a character mod q.
    
    Uses multiple LMFDB endpoints to find zeros.
    """
    zeros = []
    
    # Method 1: Direct L-function lookup
    label = f"character/dirichlet/{modulus}/{char_index}"
    
    # Try lfunc_lfunctions endpoint
    url = f"{LMFDB_BASE}/lfunc_lfunctions/?label={modulus}.{char_index}&_format=json"
    try:
        response = requests.get(url, timeout=30)
        if response.status_code == 200:
            data = response.json()
            results = data.get('data', [])
            for item in results:
                if 'positive_zeros' in item:
                    zeros = item['positive_zeros'][:max_zeros]
                    return zeros
                if 'z1' in item:  # First zero
                    zeros.append(item['z1'])
    except:
        pass
    
    # Method 2: Try character Dirichlet endpoint
    url = f"{LMFDB_BASE}/char_dir_values/?modulus={modulus}&_format=json&_limit=1"
    try:
        response = requests.get(url, timeout=30)
        if response.status_code == 200:
            data = response.json()
            # Look for associated L-function
            pass
    except:
        pass
    
    return zeros


def search_lmfdb_zeros(conductor: int, max_zeros: int = 100) -> Tuple[List[float], str]:
    """
    Search LMFDB for any L-function zeros with given conductor.
    Returns (zeros, source_description).
    """
    
    # Try multiple approaches
    
    # 1. Search lfunc_lfunctions by conductor
    url = f"{LMFDB_BASE}/lfunc_lfunctions/?conductor={conductor}&_format=json&_limit=5"
    try:
        response = requests.get(url, timeout=30)
        if response.status_code == 200:
            data = response.json()
            results = data.get('data', [])
            for item in results:
                if 'positive_zeros' in item and item['positive_zeros']:
                    return item['positive_zeros'][:max_zeros], f"lfunc conductor={conductor}"
    except Exception as e:
        print(f"  API error for conductor {conductor}: {e}")
    
    # 2. Try Dirichlet character L-functions
    # For prime conductor q, there are φ(q)-1 primitive characters
    for chi_idx in range(2, min(conductor, 10)):
        zeros = get_dirichlet_lfunc_zeros(conductor, chi_idx, max_zeros)
        if zeros:
            return zeros, f"Dirichlet χ_{conductor}.{chi_idx}"
    
    return [], "not found"


print("LMFDB API functions defined")

## 2. Test LMFDB Connection

In [ ]:
# Test API connection
print("Testing LMFDB API connection...\n")

# Test 1: Get some L-function data
test_url = f"{LMFDB_BASE}/lfunc_lfunctions/?conductor=5&_format=json&_limit=3"
response = requests.get(test_url, timeout=30)
print(f"Status: {response.status_code}")

if response.status_code == 200:
    data = response.json()
    print(f"Found {len(data.get('data', []))} L-functions with conductor 5")
    
    # Show available fields
    if data.get('data'):
        print(f"\nAvailable fields: {list(data['data'][0].keys())[:15]}...")
        
        # Check for zeros
        item = data['data'][0]
        if 'positive_zeros' in item:
            print(f"\n✓ Found zeros! First few: {item['positive_zeros'][:5]}")
        elif 'z1' in item:
            print(f"\n✓ Found first zero: z1 = {item['z1']}")
        else:
            print("\n⚠ No zeros in this response, trying different endpoint...")
else:
    print(f"API returned status {response.status_code}")

In [ ]:
# Try to get actual zeros - explore what's available
print("Exploring LMFDB for L-function zeros...\n")

# Check lfunc_zeros endpoint
url = f"{LMFDB_BASE}/lfunc_zeros/?_format=json&_limit=5"
try:
    response = requests.get(url, timeout=30)
    print(f"lfunc_zeros endpoint: {response.status_code}")
    if response.status_code == 200:
        data = response.json()
        print(f"  Records: {len(data.get('data', []))}")
        if data.get('data'):
            print(f"  Fields: {list(data['data'][0].keys())}")
except Exception as e:
    print(f"  Error: {e}")

# Check for Riemann zeta zeros specifically
print("\nLooking for Riemann zeta zeros...")
url = f"{LMFDB_BASE}/lfunc_lfunctions/?label=1-1-1.1-r0-0-0&_format=json"
try:
    response = requests.get(url, timeout=30)
    if response.status_code == 200:
        data = response.json()
        if data.get('data'):
            item = data['data'][0]
            print(f"  Label: {item.get('label', 'N/A')}")
            if 'positive_zeros' in item:
                zs = item['positive_zeros']
                print(f"  ✓ Has {len(zs)} zeros!")
                print(f"  First 10: {zs[:10]}")
except Exception as e:
    print(f"  Error: {e}")

## 3. Fetch Zeros for Test Conductors

In [ ]:
# Define conductors to test
GIFT_CONDUCTORS = [7, 8, 11, 13, 14, 21, 27, 77, 99]
NON_GIFT_CONDUCTORS = [6, 9, 10, 15, 16, 17, 19, 23, 25]

ALL_CONDUCTORS = sorted(GIFT_CONDUCTORS + NON_GIFT_CONDUCTORS)

print(f"GIFT conductors: {GIFT_CONDUCTORS}")
print(f"Non-GIFT conductors: {NON_GIFT_CONDUCTORS}")

In [ ]:
# Fetch zeros for each conductor
print("Fetching L-function zeros from LMFDB...\n")

conductor_zeros = {}

for q in ALL_CONDUCTORS:
    is_gift = "★" if q in GIFT_CONDUCTORS else " "
    print(f"{is_gift} Conductor {q:3d}: ", end="")
    
    zeros, source = search_lmfdb_zeros(q, max_zeros=200)
    
    if zeros:
        conductor_zeros[q] = zeros
        print(f"✓ {len(zeros)} zeros from {source}")
    else:
        print(f"✗ No zeros found")
    
    time.sleep(0.5)  # Be nice to the API

print(f"\nTotal conductors with zeros: {len(conductor_zeros)}")

## 4. Alternative: Use Pre-computed Zeros from LMFDB Website

If the API doesn't return zeros directly, we can try the web interface or use known databases.

In [ ]:
# Alternative: Fetch from LMFDB web pages (scraping as backup)
def fetch_zeros_from_webpage(conductor: int) -> List[float]:
    """
    Try to fetch zeros from LMFDB web interface.
    """
    # LMFDB L-function page format
    # For Dirichlet: https://www.lmfdb.org/L/1/conductor/character_orbit/
    
    # Try known patterns
    urls_to_try = [
        f"https://www.lmfdb.org/L/download_zeros/1/{conductor}/a/",
        f"https://www.lmfdb.org/api/lfunc_lfunctions/?degree=1&conductor={conductor}&_format=json",
    ]
    
    for url in urls_to_try:
        try:
            response = requests.get(url, timeout=30)
            if response.status_code == 200:
                # Try to parse as JSON
                try:
                    data = response.json()
                    if 'data' in data and data['data']:
                        for item in data['data']:
                            if 'positive_zeros' in item:
                                return item['positive_zeros']
                except:
                    # Try as plain text (zeros list)
                    lines = response.text.strip().split('\n')
                    zeros = [float(line) for line in lines if line.strip()]
                    if zeros:
                        return zeros
        except:
            continue
    
    return []

# Test on one conductor
print("Testing webpage fetch for conductor 7...")
test_zeros = fetch_zeros_from_webpage(7)
print(f"  Found {len(test_zeros)} zeros" if test_zeros else "  No zeros found via webpage")

In [ ]:
# If LMFDB API doesn't give zeros, use Odlyzko's tables as reference
# and scale them by conductor (this is an approximation for testing)

def get_riemann_zeros_from_web(num_zeros: int = 100) -> List[float]:
    """
    Get Riemann zeta zeros from a web source.
    """
    # Andrew Odlyzko's tables
    url = "https://www-users.cse.umn.edu/~odlyzko/zeta_tables/zeros1"
    try:
        response = requests.get(url, timeout=30)
        if response.status_code == 200:
            lines = response.text.strip().split('\n')
            zeros = []
            for line in lines[:num_zeros]:
                try:
                    zeros.append(float(line.strip()))
                except:
                    continue
            return zeros
    except Exception as e:
        print(f"Error fetching Odlyzko zeros: {e}")
    return []

print("Fetching Riemann zeros from Odlyzko tables...")
riemann_zeros = get_riemann_zeros_from_web(500)
print(f"Got {len(riemann_zeros)} Riemann zeros")
if riemann_zeros:
    print(f"First 5: {riemann_zeros[:5]}")

## 5. Recurrence Analysis Functions

In [ ]:
GIFT_LAGS = [5, 8, 13, 27]
STANDARD_LAGS = [1, 2, 3, 4]

def fit_recurrence(zeros: List[float], lags: List[int]) -> Tuple[List[float], float]:
    """
    Fit recurrence: γ_n = Σ_k a_k γ_{n-lag_k} + c
    Returns (coefficients, relative_error_percent).
    """
    if len(zeros) < max(lags) + 10:
        return [], float('inf')
    
    max_lag = max(lags)
    n_points = len(zeros) - max_lag
    
    # Design matrix
    X = np.zeros((n_points, len(lags) + 1))
    y = np.zeros(n_points)
    
    for i in range(n_points):
        n = i + max_lag
        y[i] = zeros[n]
        for j, lag in enumerate(lags):
            X[i, j] = zeros[n - lag]
        X[i, -1] = 1  # constant
    
    try:
        coeffs, residuals, rank, s = np.linalg.lstsq(X, y, rcond=None)
        y_fit = X @ coeffs
        rmse = np.sqrt(np.mean((y - y_fit)**2))
        rel_error = rmse / np.mean(np.abs(y)) * 100
        return coeffs.tolist(), rel_error
    except:
        return [], float('inf')


def compute_fibonacci_R(coeffs: List[float], lags: List[int]) -> float:
    """
    Fibonacci constraint: R = (8 × a_8) / (13 × a_13)
    Should be close to 1 for GIFT structure.
    """
    if len(coeffs) < 3 or 8 not in lags or 13 not in lags:
        return float('nan')
    
    idx_8 = lags.index(8)
    idx_13 = lags.index(13)
    
    a_8 = coeffs[idx_8]
    a_13 = coeffs[idx_13]
    
    if abs(13 * a_13) < 1e-10:
        return float('nan')
    
    return (8 * a_8) / (13 * a_13)


print("Recurrence functions defined")

## 6. Run Selectivity Test

In [ ]:
# If we have actual L-function zeros from LMFDB, use them
# Otherwise, fall back to Riemann zeros (noting this is a proxy)

use_proxy = len(conductor_zeros) < 5  # If we got < 5 conductors from LMFDB

if use_proxy:
    print("⚠️  Using Riemann zeros as proxy (LMFDB data limited)")
    print("   Results are indicative only, not definitive.\n")
    
    # Use Riemann zeros for all conductors
    if not riemann_zeros:
        print("ERROR: No Riemann zeros available either!")
    else:
        for q in ALL_CONDUCTORS:
            conductor_zeros[q] = riemann_zeros.copy()
else:
    print(f"✓ Using actual L-function zeros from LMFDB ({len(conductor_zeros)} conductors)\n")

In [ ]:
# Run the selectivity test
print("=" * 70)
print("CONDUCTOR SELECTIVITY TEST")
print("=" * 70)
print(f"{'★ = GIFT conductor':40s}")
print(f"\n{'q':>4} | {'GIFT err%':>10} | {'Std err%':>10} | {'Fib R':>10} | {'|R-1|':>8}")
print("-" * 60)

results = []

for q in ALL_CONDUCTORS:
    if q not in conductor_zeros or not conductor_zeros[q]:
        continue
    
    zeros = conductor_zeros[q]
    is_gift = q in GIFT_CONDUCTORS
    marker = "★" if is_gift else " "
    
    # Fit recurrences
    gift_coeffs, gift_err = fit_recurrence(zeros, GIFT_LAGS)
    std_coeffs, std_err = fit_recurrence(zeros, STANDARD_LAGS)
    
    # Fibonacci R
    fib_R = compute_fibonacci_R(gift_coeffs, GIFT_LAGS)
    R_dev = abs(fib_R - 1) if not np.isnan(fib_R) else float('nan')
    
    results.append({
        'conductor': q,
        'is_gift': is_gift,
        'gift_error': gift_err,
        'std_error': std_err,
        'fibonacci_R': fib_R,
        'R_deviation': R_dev,
        'gift_coeffs': gift_coeffs
    })
    
    print(f"{marker}{q:3d} | {gift_err:>10.4f} | {std_err:>10.4f} | {fib_R:>10.4f} | {R_dev:>8.4f}")

print("-" * 60)

In [ ]:
# Summary statistics
print("\n" + "=" * 70)
print("SUMMARY STATISTICS")
print("=" * 70)

gift_results = [r for r in results if r['is_gift']]
non_gift_results = [r for r in results if not r['is_gift']]

# Extract R deviations
gift_R_devs = [r['R_deviation'] for r in gift_results if not np.isnan(r['R_deviation'])]
non_gift_R_devs = [r['R_deviation'] for r in non_gift_results if not np.isnan(r['R_deviation'])]

print(f"\nGIFT conductors (n={len(gift_results)}):")
if gift_R_devs:
    print(f"  Mean |R - 1|: {np.mean(gift_R_devs):.4f}")
    print(f"  Std |R - 1|:  {np.std(gift_R_devs):.4f}")

print(f"\nNon-GIFT conductors (n={len(non_gift_results)}):")
if non_gift_R_devs:
    print(f"  Mean |R - 1|: {np.mean(non_gift_R_devs):.4f}")
    print(f"  Std |R - 1|:  {np.std(non_gift_R_devs):.4f}")

# Statistical test
if len(gift_R_devs) >= 3 and len(non_gift_R_devs) >= 3:
    t_stat, p_value = stats.ttest_ind(gift_R_devs, non_gift_R_devs)
    print(f"\nStatistical test (t-test):")
    print(f"  t-statistic: {t_stat:.4f}")
    print(f"  p-value: {p_value:.4f}")
    
    if p_value < 0.05:
        print("  → Significant difference (p < 0.05)!")
    else:
        print("  → No significant difference (p ≥ 0.05)")

In [ ]:
# Verdict
print("\n" + "=" * 70)
print("VERDICT")
print("=" * 70)

if gift_R_devs and non_gift_R_devs:
    mean_gift = np.mean(gift_R_devs)
    mean_non = np.mean(non_gift_R_devs)
    
    if mean_gift < mean_non:
        print(f"\n✓ GIFT conductors have |R-1| = {mean_gift:.4f} < {mean_non:.4f}")
        print("  → CONSISTENT with GIFT hypothesis (Fibonacci constraint tighter)")
    else:
        print(f"\n✗ Non-GIFT conductors have |R-1| = {mean_non:.4f} < {mean_gift:.4f}")
        print("  → NOT consistent with GIFT selectivity")

if use_proxy:
    print("\n⚠️  CAVEAT: Using proxy data (scaled Riemann zeros).")
    print("   This test should be repeated with real LMFDB L-function zeros.")

## 7. Visualization

In [ ]:
# Plot results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: |R - 1| for each conductor
ax1 = axes[0]
gift_qs = [r['conductor'] for r in gift_results]
gift_Rs = [r['R_deviation'] for r in gift_results]
non_gift_qs = [r['conductor'] for r in non_gift_results]
non_gift_Rs = [r['R_deviation'] for r in non_gift_results]

ax1.scatter(gift_qs, gift_Rs, c='blue', s=100, label='GIFT', marker='*')
ax1.scatter(non_gift_qs, non_gift_Rs, c='red', s=60, label='Non-GIFT', marker='o')
ax1.axhline(y=0, color='green', linestyle='--', alpha=0.5, label='Perfect R=1')
ax1.set_xlabel('Conductor q')
ax1.set_ylabel('|R - 1| (deviation from Fibonacci constraint)')
ax1.set_title('Fibonacci Constraint by Conductor')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Box plot comparison
ax2 = axes[1]
data_to_plot = [gift_R_devs, non_gift_R_devs]
bp = ax2.boxplot(data_to_plot, labels=['GIFT', 'Non-GIFT'], patch_artist=True)
bp['boxes'][0].set_facecolor('lightblue')
bp['boxes'][1].set_facecolor('lightcoral')
ax2.set_ylabel('|R - 1|')
ax2.set_title('Distribution of Fibonacci Constraint Deviation')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('conductor_selectivity_lmfdb.png', dpi=150)
plt.show()

print("\n✓ Plot saved as 'conductor_selectivity_lmfdb.png'")

## 8. Export Results

In [ ]:
# Export to JSON
output = {
    'use_proxy_data': use_proxy,
    'gift_conductors': GIFT_CONDUCTORS,
    'non_gift_conductors': NON_GIFT_CONDUCTORS,
    'results': results,
    'summary': {
        'gift_mean_R_deviation': float(np.mean(gift_R_devs)) if gift_R_devs else None,
        'non_gift_mean_R_deviation': float(np.mean(non_gift_R_devs)) if non_gift_R_devs else None,
        'gift_is_better': bool(np.mean(gift_R_devs) < np.mean(non_gift_R_devs)) if gift_R_devs and non_gift_R_devs else None
    }
}

# Convert numpy types
def convert(obj):
    if isinstance(obj, (np.integer, np.floating, np.bool_)):
        return obj.item()
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, dict):
        return {k: convert(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [convert(x) for x in obj]
    return obj

with open('lmfdb_selectivity_results.json', 'w') as f:
    json.dump(convert(output), f, indent=2)

print("✓ Results saved to 'lmfdb_selectivity_results.json'")

# Download in Colab
try:
    from google.colab import files
    files.download('lmfdb_selectivity_results.json')
    files.download('conductor_selectivity_lmfdb.png')
except:
    print("(Not in Colab - files saved locally)")

---

## Summary

This notebook tests the **conductor selectivity hypothesis**:

> GIFT conductors (7, 8, 11, 13, 14, 21, 27, 77, 99) should show tighter Fibonacci constraint R ≈ 1 in the [5, 8, 13, 27] recurrence of their L-function zeros.

**Key metric**: |R - 1| where R = (8 × a₈) / (13 × a₁₃)

Lower |R - 1| → closer to Fibonacci structure → supports GIFT hypothesis.

---

*GIFT Framework — LMFDB Selectivity Test*  
*February 2026*